In [1]:
import pandas as pd
from scipy import stats
from sklearn.metrics import roc_auc_score, matthews_corrcoef
import numpy as np
import os

In [5]:
# This should be changed per assay
base_dir='/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results/'
model='Nucleotide Transformer'
score_column = 'NT_scores'
dataset_result_extension='.csv'

In [15]:
# read in the reference wt sequences
wt_seqs = pd.read_csv('/n/groups/marks/projects/RNAgym/mutational_assays/reference_sheet_cleaned_CAS.csv', encoding='latin-1')
wt_seqs.dropna(inplace=True)
wt_seqs['Year'] = wt_seqs['Year'].astype(int)
wt_seqs['First Author Last Name'] = wt_seqs['First Author Last Name'].astype(str)
wt_seqs['Molecule Type'] = wt_seqs['Molecule Type'].astype(str)
wt_seqs.drop(['Unnamed: 0', 'index'], inplace=True, axis=1)

In [16]:

spearmans_single = []
aucs_single = []
mccs_single = []

spearmans_multiple = []
aucs_multiple = []
mccs_multiple = []

spearmans_combined = []
aucs_combined = []
mccs_combined = []

for _, row in wt_seqs.iterrows():
    dataset = row['fitness filename']
    df_path = f'{base_dir}/{dataset}{dataset_result_extension}'
    if os.path.exists(df_path):
        df = pd.read_csv(df_path)
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
#         df.drop(['Unnamed: 0'], axis=1, inplace=True)
        assay_col = df.columns[1]
        mutation_column = 'mutant' if 'mutant' in df.columns else 'mutation' if 'mutation' in df.columns else 'mutations' if 'mutations' in df.columns else None
        
        if mutation_column:
            df['mutation_count'] = df[mutation_column].apply(lambda x: len(x.split(',')))
            
            single_mutation_df = df[df['mutation_count'] == 1]
            multiple_mutation_df = df[df['mutation_count'] > 1]
            
            # Single mutations
            if not single_mutation_df.empty:
                assay_scores_single = np.array(single_mutation_df[assay_col])
                model_stat_single = np.array(single_mutation_df[score_column])
                
                spearman_corr_single = stats.spearmanr(assay_scores_single, model_stat_single).correlation
                spearmans_single.append(abs(spearman_corr_single))
                
                auc_single = roc_auc_score((assay_scores_single > np.median(assay_scores_single)).astype(int), model_stat_single)
                aucs_single.append(auc_single)
                
                mcc_single = matthews_corrcoef((assay_scores_single > np.median(assay_scores_single)).astype(int), (model_stat_single > np.median(model_stat_single)).astype(int))
                mccs_single.append(mcc_single)
            else:
                spearmans_single.append(np.nan)
                aucs_single.append(np.nan)
                mccs_single.append(np.nan)
            
            # Multiple mutations
            if not multiple_mutation_df.empty:
                assay_scores_multiple = np.array(multiple_mutation_df[assay_col])
                model_stat_multiple = np.array(multiple_mutation_df[score_column])
                
                spearman_corr_multiple = stats.spearmanr(assay_scores_multiple, model_stat_multiple).correlation
                spearmans_multiple.append(abs(spearman_corr_multiple))
                
                auc_multiple = roc_auc_score((assay_scores_multiple > np.median(assay_scores_multiple)).astype(int), model_stat_multiple)
                aucs_multiple.append(auc_multiple)
                
                mcc_multiple = matthews_corrcoef((assay_scores_multiple > np.median(assay_scores_multiple)).astype(int), (model_stat_multiple > np.median(model_stat_multiple)).astype(int))
                mccs_multiple.append(mcc_multiple)
            else:
                spearmans_multiple.append(np.nan)
                aucs_multiple.append(np.nan)
                mccs_multiple.append(np.nan)
                
            # Combined
            assay_scores_combined = np.array(df[assay_col])
            model_stat_combined = np.array(df[score_column])

            spearman_corr_combined = stats.spearmanr(assay_scores_combined, model_stat_combined).correlation
            spearmans_combined.append(abs(spearman_corr_combined))

            auc_combined = roc_auc_score((assay_scores_combined > np.median(assay_scores_combined)).astype(int), model_stat_combined)
            aucs_combined.append(auc_combined)

            mcc_combined = matthews_corrcoef((assay_scores_combined > np.median(assay_scores_combined)).astype(int), (model_stat_combined > np.median(model_stat_combined)).astype(int))
            mccs_combined.append(mcc_combined)
        else:
            spearmans_single.append(np.nan)
            aucs_single.append(np.nan)
            mccs_single.append(np.nan)
            spearmans_multiple.append(np.nan)
            aucs_multiple.append(np.nan)
            mccs_multiple.append(np.nan)
            spearmans_combined.append(np.nan)
            aucs_combined.append(np.nan)
            mccs_combined.append(np.nan)
    else:
        spearmans_single.append(np.nan)
        aucs_single.append(np.nan)
        mccs_single.append(np.nan)
        spearmans_multiple.append(np.nan)
        aucs_multiple.append(np.nan)
        mccs_multiple.append(np.nan)
        spearmans_combined.append(np.nan)
        aucs_combined.append(np.nan)
        mccs_combined.append(np.nan)

# Group by type and calculate averages
types = ['mRNA', 'tRNA', 'Aptamer', 'Ribozyme', 'All']
wt_seqs['type'] = wt_seqs['fitness filename'].apply(lambda x: x.split('_')[-1])

# Append new columns to wt_seqs for single and multiple mutations
wt_seqs['Spearmans_single'] = spearmans_single
wt_seqs['AUC_single'] = aucs_single
wt_seqs['MCC_single'] = mccs_single

wt_seqs['Spearmans_multiple'] = spearmans_multiple
wt_seqs['AUC_multiple'] = aucs_multiple
wt_seqs['MCC_multiple'] = mccs_multiple

wt_seqs['Spearmans_combined'] = spearmans_combined
wt_seqs['AUC_combined'] = aucs_combined
wt_seqs['MCC_combined'] = mccs_combined

# Calculate averages for each metric
metrics = ['Spearmans', 'AUC', 'MCC']
average_metrics_single = {}
average_metrics_multiple = {}
average_metrics_combined = {}

for metric in metrics:
    single_metric_col = f'{metric}_single'
    multiple_metric_col = f'{metric}_multiple'
    combined_metric_col = f'{metric}_combined'
    
    type_groups_single = wt_seqs.groupby('type')[single_metric_col].mean().reset_index()
    type_groups_single.loc[len(type_groups_single.index)] = ['All', wt_seqs[single_metric_col].mean()]
    averages_single = {type_: type_groups_single[type_groups_single['type'].str.contains(type_, case=False)][single_metric_col].mean() for type_ in types}
    average_metrics_single[metric] = averages_single
    
    type_groups_multiple = wt_seqs.groupby('type')[multiple_metric_col].mean().reset_index()
    type_groups_multiple.loc[len(type_groups_multiple.index)] = ['All', wt_seqs[multiple_metric_col].mean()]
    averages_multiple = {type_: type_groups_multiple[type_groups_multiple['type'].str.contains(type_, case=False)][multiple_metric_col].mean() for type_ in types}
    average_metrics_multiple[metric] = averages_multiple
    
    type_groups_combined = wt_seqs.groupby('type')[combined_metric_col].mean().reset_index()
    type_groups_combined.loc[len(type_groups_combined.index)] = ['All', wt_seqs[combined_metric_col].mean()]
    averages_combined = {type_: type_groups_combined[type_groups_combined['type'].str.contains(type_, case=False)][combined_metric_col].mean() for type_ in types}
    average_metrics_combined[metric] = averages_combined

# Create final result DataFrame
result_df_single = pd.DataFrame(average_metrics_single)
result_df_single.insert(0, 'Type', types)

result_df_multiple = pd.DataFrame(average_metrics_multiple)
result_df_multiple.insert(0, 'Type', types)

result_df_combined = pd.DataFrame(average_metrics_combined)
result_df_combined.insert(0, 'Type', types)

with pd.option_context("display.precision", 3):
    print("Single Mutations Metrics:\n", result_df_single)
    print("Multiple Mutations Metrics:\n", result_df_multiple)
    print("Combined Mutations Metrics:\n", result_df_combined)

Single Mutations Metrics:
               Type  Spearmans    AUC    MCC
mRNA          mRNA      0.199  0.400 -0.138
tRNA          tRNA      0.165  0.491  0.016
Aptamer    Aptamer      0.105  0.438 -0.092
Ribozyme  Ribozyme      0.126  0.504  0.019
All            All      0.138  0.484 -0.009
Multiple Mutations Metrics:
               Type  Spearmans    AUC    MCC
mRNA          mRNA      0.246  0.376 -0.177
tRNA          tRNA      0.094  0.552  0.078
Aptamer    Aptamer      0.145  0.463 -0.071
Ribozyme  Ribozyme      0.107  0.488 -0.019
All            All      0.124  0.481 -0.032
Combined Mutations Metrics:
               Type  Spearmans    AUC    MCC
mRNA          mRNA      0.245  0.375 -0.177
tRNA          tRNA      0.099  0.553  0.079
Aptamer    Aptamer      0.159  0.454 -0.079
Ribozyme  Ribozyme      0.105  0.488 -0.019
All            All      0.126  0.479 -0.033


In [17]:
# read in the reference wt sequences
wt_seqs = pd.read_csv('/n/groups/marks/projects/RNAgym/mutational_assays/reference_sheet_cleaned_CAS.csv', encoding='latin-1')
wt_seqs.dropna(inplace=True)
wt_seqs['Year'] = wt_seqs['Year'].astype(int)
wt_seqs['First Author Last Name'] = wt_seqs['First Author Last Name'].astype(str)
wt_seqs['Molecule Type'] = wt_seqs['Molecule Type'].astype(str)
wt_seqs.drop(['Unnamed: 0', 'index'], inplace=True, axis=1)

spearmans = []
aucs = []
mccs = []

for _, row in wt_seqs.iterrows():
    dataset = row['fitness filename']
    df_path = f'{base_dir}/{dataset}{dataset_result_extension}'
    print(df_path)
    if os.path.exists(df_path):
        df = pd.read_csv(df_path)
        df.drop(['Unnamed: 0'], axis=1, inplace=True)
        assay_col = df.columns[1]
        
        assay_scores = np.array(df[assay_col])
        model_stat = np.array(df[score_column])
        
        # Spearman correlation
        spearman_corr = stats.spearmanr(assay_scores, model_stat).correlation
        spearmans.append(abs(spearman_corr))

        # AUC
        auc = roc_auc_score((assay_scores > np.median(assay_scores)).astype(int), model_stat)
        aucs.append(auc)

        # MCC
        mcc = matthews_corrcoef((assay_scores > np.median(assay_scores)).astype(int), (model_stat > np.median(model_stat)).astype(int))
        mccs.append(mcc)
    else:
        spearmans.append(np.nan)
        aucs.append(np.nan)
        mccs.append(np.nan)

# Append new columns to wt_seqs
wt_seqs['Spearman'] = spearmans
wt_seqs['AUC'] = aucs
wt_seqs['MCC'] = mccs

# Calculate averages for each metric
# metrics = ['Spearman', 'AUC', 'MCC']
# average_metrics = {}

# for metric in metrics:
#     type_groups = wt_seqs.groupby('type')[metric].mean().reset_index()
#     type_groups.loc[len(type_groups.index)] = ['All', wt_seqs[metric].mean()]
#     averages = {type_: type_groups[type_groups['type'].str.contains(type_, case=False)][metric].mean() for type_ in types}
#     average_metrics[metric] = averages

# # Create final result DataFrame
# result_df = pd.DataFrame(average_metrics)
# result_df.insert(0, 'Type', types)

# with pd.option_context("display.precision", 3):
#     print("Assay Metrics:\n", result_df)

/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//ke_2017_mRNA.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//soo_2021_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Andreasson_2020_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Kobori_2015_tw_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Kobori_2015_j12_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Kobori_2015_p4_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Guy_2014_tRNA.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Kobori_2018_ribozyme.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Townshend_2015_4nt_aptamer.csv
/n/groups/marks/projects/RNAgym/baselines/Nucleotide_Transformer/results//Townshend_2015_5nt_aptamer.csv
/n

In [18]:
wt_seqs

,Title,First Author Last Name,Year,Molecule Type,Raw Construct Sequence,fitness filename,Spearman,AUC,MCC
0,Saturation mutagenesis reveals manifold determ...,Ke,2017,mRNA,ccccacctcttcttcttttctagAGTTGCTGCTGGGAGCTCCAGCA...,ke_2017_mRNA,0.383205,0.303543,-0.279279
1,Fitness landscape of a dynamic RNA structure,Soo,2021,ribozyme,CTCTCTAGGTAGCAATATTACGACATGGAGGGAAAAGTTATCAGGC...,soo_2021_ribozyme,0.009862,0.496602,-0.008671
2,Comprehensive sequence-to-function mapping of ...,Andreasson,2020,ribozyme riboswitch,ATAAAAGCGCCAGAACTACAGAAATGTAGTTGACGAGGAGGAGGTT...,Andreasson_2020_ribozyme,0.114690,0.447871,-0.065024
3,High-throughput assay and engineering of self-...,Kobori,2015,Lib-Tw,TAATACGACTCACTATAGGGCGCGGCATTAATGCAGCTTTATTGGA...,Kobori_2015_tw_ribozyme,0.018681,0.495067,-0.007264
4,High-throughput assay and engineering of self-...,Kobori,2015,Lib-J1/2,TAATACGACTCACTATAGGGCCGCGACTCTAGAAGTGATGCTCTGC...,Kobori_2015_j12_ribozyme,0.229690,0.620117,0.140625
5,High-throughput assay and engineering of self-...,Kobori,2015,Lib-P4,TAATACGACTCACTATAGGGCCGCGACTCTAGAAGTGATGCTCTGC...,Kobori_2015_p4_ribozyme,0.004775,0.468445,-0.046875
6,Identification of the determinants of tRNA fun...,Guy,2014,tRNA,GATCTAACAAAGTTCATAAAGAAATTACTCTCGGTAGCCAAGTTGG...,Guy_2014_tRNA,0.046266,0.533315,0.064516
7,Deep sequencing analysis of aptazyme variants ...,Kobori,2018,pistol ribozyme,TAATACGACTCACTATAGGGTCGNNNNNNNATAATCGCGTGGATAT...,Kobori_2018_ribozyme,0.084779,0.459204,-0.052490
8,High-throughput cellular RNA device engineering,Townshend,2015,4nt RNA aptamer,GCTGTCACCGGANNNNNNNNNTCNNNTNNNTNCNNNNNNNNNTCNN...,Townshend_2015_4nt_aptamer,0.093221,0.533514,-0.004854
9,High-throughput cellular RNA device engineering,Townshend,2015,5nt RNA aptamer,GCTGTCACCGGANNNNNNNCNNNNTNNTGANNCCNTNNNNNNNNCG...,Townshend_2015_5nt_aptamer,0.220651,0.374713,-0.200563
